<a href="https://colab.research.google.com/github/ahmedhas32/Data-science-projects/blob/master/mapping%20arabic%20names%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import xgboost
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from scipy.stats import skew
from collections import OrderedDict
from sklearn import preprocessing
from datetime import datetime


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from difflib import SequenceMatcher
from difflib import get_close_matches 


def sim_metric(df, col1, col2):
           metric=None
           try : metric = SequenceMatcher(None, df[col1], df[col2]).ratio()
           except TypeError: metric=0 
           return metric

In [ ]:
import re
def remove_prefix(text, prefix):
    return re.sub(r'^{0}'.format(re.escape(prefix)), '', text)

In [ ]:
!pip install fsspec>=0.3.3

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
dwh_city = pd.read_excel('/content/SAMA_Mapping.xlsx' , sheet_name='DWH_City')

In [ ]:
pos_dest = pd.read_excel('/content/SAMA_Mapping.xlsx' , sheet_name='POS_Dest')

In [ ]:
dwh_city.dropna(subset=['CITY_NAME_EN'] , inplace=True)

In [ ]:
dwh_city['CITY_NAME_EN']=dwh_city['CITY_NAME_EN'].str.lower()

In [ ]:
dwh_city['CITY_NAME_EN'] = dwh_city['CITY_NAME_EN'].replace('al madinah al munawwarah' , 'madina').replace('makkah al mukarramah' , 'makkah')

In [ ]:
dwh_city['CITY_NAME_EN'] =dwh_city['CITY_NAME_EN'] .apply(lambda x: remove_prefix(x , r'a[a-z]\s')).apply(lambda x:remove_prefix(x ,  r'a[a-z][a-z]\s' )).apply(lambda x: x.strip("'"))

In [ ]:
dwh_city['GOVERNORATE_NAME_EN']=dwh_city['GOVERNORATE_NAME_EN'].str.lower()

In [ ]:
pos_dest['Destination_City']=pos_dest['Destination_City'].str.lower()

In [ ]:
saudi_cities=dwh_city.loc[dwh_city['Country'] == 'Saudi Arabia' , ['CITY_CD' , 'CITY_NAME_EN' , 'PROVINCE_NAME_EN' , 'GOVERNORATE_NAME_EN']].drop_duplicates()

saudi_cities['CITY_NAME_EN']=saudi_cities['CITY_NAME_EN']


In [ ]:
city_names=list(set(saudi_cities['CITY_NAME_EN'].dropna()) )
city_names1=list(set(dwh_city['GOVERNORATE_NAME_EN'].dropna()))

In [ ]:
pos_dest=pos_dest.loc[pd.isna(pos_dest['Destination_City'])==0, :]
pos_dest.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 188 entries, 0 to 187
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Destination_Region  188 non-null    object 
 1   Destination_City    188 non-null    object 
 2   CITY_CD             0 non-null      float64
 3   CITY_NAME_EN        0 non-null      float64
dtypes: float64(2), object(2)
memory usage: 7.3+ KB


In [ ]:
pos_dest['Destination_City']=pos_dest['Destination_City'].apply(lambda x: remove_prefix(x , r'a[a-z]\s')).apply(lambda x:remove_prefix(  x, r'a[a-z][a-z]\s'))

In [ ]:
pos_dest['closest'] = pos_dest['Destination_City'].apply(lambda x : get_close_matches(x,city_names))

In [ ]:
pos_dest = pos_dest.reset_index().explode(column='closest')


In [ ]:
pos_dest_done=pos_dest.drop(columns=['CITY_CD' ,'CITY_NAME_EN']).merge(how = 'left' ,
    right = saudi_cities , left_on = 'closest' , right_on= 'CITY_NAME_EN') 

In [ ]:
pos_dest_done['a_c_comp'] = pos_dest_done.apply(sim_metric,
                          args=('Destination_Region', 'PROVINCE_NAME_EN'),
                          axis=1) * pos_dest_done.apply(sim_metric,
                          args=('Destination_City', 'CITY_NAME_EN'),
                          axis=1) 





In [ ]:
#pos_dest_done['a_e_comp'] =  pos_dest_done.apply(sim_metric,
#                          args=('Destination_Region', 'PROVINCE_NAME_EN'),
#                          axis=1) * pos_dest_done.apply(sim_metric,
#                          args=('Destination_City', 'CITY_NAME_EN'),
#                          axis=1)* pos_dest_done.apply(sim_metric,
#                          args=('Destination_City', 'GOVERNORATE_NAME_EN'),
#                          axis=1)


In [ ]:
final_pos=pos_dest_done.loc[(pos_dest_done.groupby('Destination_City')['a_c_comp'].idxmax()) , ['index','Destination_Region','Destination_City','CITY_CD','CITY_NAME_EN','PROVINCE_NAME_EN','a_c_comp']
                  ].sort_values(by=['index']).loc[pos_dest_done['a_c_comp'] >= 0.4]


In [ ]:
final_pos['index'] +=1

In [ ]:
final_pos

,index,Destination_Region,Destination_City,CITY_CD,CITY_NAME_EN,PROVINCE_NAME_EN,a_c_comp
2,3,Bahah,mukhwah,3236,al mukhwah,Albaha,0.449198
6,4,Eastern,jubail,113,al jubail,Eastern,0.800000
11,5,Ha'il,shamly,2280,sahayy,Hail,0.592593
12,6,Jizan,jizan,17,jazan,Jazan,0.640000
15,7,Makkah,jeddah,18,jeddah,Makkah,1.000000
...,...,...,...,...,...,...,...
547,181,Asir,shaabain,1278,sa'aban,Aseer,0.533333
549,182,Eastern,hafr albatin,47,hafar al batin,Eastern,0.923077
552,183,Jizan,abo araish,3525,abu arish,Jazan,0.673684
555,184,Makkah,bani saad,2202,bani da'ad,Makkah,0.842105


In [ ]:
final_pos.to_excel('pos_dest.xlsx')

In [ ]:
pos_dest1 = pos_dest_done.loc[(pos_dest_done.groupby('Destination_City')['a_c_comp'].idxmax()) , ['index','Destination_Region','Destination_City','CITY_CD','CITY_NAME_EN','PROVINCE_NAME_EN','a_c_comp']
                  ].sort_values(by=['index']).loc[pos_dest_done['a_c_comp'] < 0.40, ['index' ,	'Destination_Region' ,	'Destination_City']]


In [ ]:
pos_dest1['closest'] = pos_dest1['Destination_City'].apply(lambda x : get_close_matches(x,city_names1))

In [ ]:
pos_dest_done1=pos_dest1.reset_index().explode(column='closest').merge(how = 'left' ,
    right = saudi_cities , left_on = 'closest' , right_on= 'GOVERNORATE_NAME_EN') 

In [ ]:
pos_dest_done1['a_d_comp'] =   pos_dest_done1.apply(sim_metric,
                          args=('Destination_Region', 'PROVINCE_NAME_EN'),
                          axis=1) * pos_dest_done1.apply(sim_metric,
                          args=('Destination_City', 'GOVERNORATE_NAME_EN'),
                          axis=1)

In [ ]:
final_pos1=pos_dest_done1.loc[(pos_dest_done1.groupby('Destination_City')['a_d_comp'].idxmax()) , ['index','Destination_Region','Destination_City','CITY_CD','CITY_NAME_EN','PROVINCE_NAME_EN', 'GOVERNORATE_NAME_EN','a_d_comp']
                  ].sort_values(by=['index']).loc[pos_dest_done1['a_d_comp'] >= 0.4]

In [ ]:
final_pos1.shape

(11, 8)

In [ ]:
final_pos1 = final_pos1.loc[ :, ['index' , 'CITY_CD' ,'GOVERNORATE_NAME_EN' , 'PROVINCE_NAME_EN' , 'a_d_comp']]

In [ ]:
final_pos1['index'] +=1

In [ ]:
final_pos1.to_excel('pos_dest1.xlsx')

In [ ]:
#non_match=set (pos_dest['Destination_City']) - set(final_pos1['Destination_City'])-set(final_pos['Destination_City'])

In [ ]:
#non_match

## Sheet 3

In [ ]:
pos_dest_final = pd.read_excel('/content/SAMA_Mapping11.xlsx' , sheet_name='POS_Dest')

In [ ]:
pos_dest_final.drop(columns = ['Unnamed: 0' , 'Destination_Region' ,'CITY_NAME_EN' ,'Updated'] , inplace=True )

In [ ]:
pos_dest_final.drop_duplicates(inplace=True)

In [ ]:
pos_dest_final.columns = ['city' , 'city_id']

In [ ]:
pos_city_country_final = pd.read_excel('/content/SAMA_Mapping11.xlsx' , sheet_name='POS_City and Country')

In [ ]:
pos_city_country_final.drop(columns = ['Unnamed: 0','Unnamed: 9' ,'Unnamed: 10' ,'Unnamed: 11' , 'CITY_NAME_EN' ,'Origin_Country','Update' ,'Country_ALPHA3_CD' , 'Country_SHORT_NAME_EN' ,'updated'] , inplace=True )

In [ ]:
pos_city_country_final.drop_duplicates(inplace=True)

In [ ]:
pos_city_country_final.columns = ['city' , 'city_id']

In [ ]:
saudi_cities=saudi_cities.loc[ : ,['CITY_NAME_EN' ,'CITY_CD' ]]


In [ ]:
saudi_cities.columns = ['city' , 'city_id']

In [ ]:
corpus = pd.concat([pos_city_country_final ,pos_dest_final ,saudi_cities])

In [ ]:
corpus.dropna(subset = ['city'] , inplace=True)

In [ ]:
corpus['city'] = corpus['city'].apply(lambda x: x.lower())

In [ ]:
corpus['city'] =corpus['city'] .replace('al madinah al munawwarah' , 'madina').replace('makkah al mukarramah' , 'makkah')

In [ ]:
corpus['city'] = corpus['city'].apply(lambda x: remove_prefix(x , 'al ')).apply(lambda x:remove_prefix(  x, 'al-'))

In [ ]:
corpus_city = list(set(corpus['city']))

In [ ]:
sama_city = pd.read_excel('/content/SAMA_Mapping11.xlsx' , sheet_name='SAMA_City')

In [ ]:
sama_city['Province'] = sama_city['Province'].str.lower()
sama_city['City'] = sama_city['City'].str.lower()

In [ ]:
sama_city.dropna(subset = ['City'] , inplace=True)

In [ ]:
sama_city['Province'] = sama_city['Province'].str.replace('region' , '')

In [ ]:
sama_city['City'] = sama_city['City'].str.replace('city' , '')
sama_city['City'] = sama_city['City'].str.replace('branch' , '')
sama_city['City'] = sama_city['City'].str.replace('east' , '')
sama_city['City'] = sama_city['City'].str.replace('north' , '')
sama_city['City'] = sama_city['City'].str.replace('west' , '')
sama_city['City'] = sama_city['City'].str.replace('south' , '')
sama_city['City'] = sama_city['City'].str.replace(' - ladies' , '')

In [ ]:
sama_city = sama_city.loc[: , ['Unnamed: 0' , 'OperationId' , 'City' , 'CITY_CD' , 'Checked']]

In [ ]:
sama_city_checked = sama_city.loc[sama_city['Checked']==1 , ['City' , 'CITY_CD']]

In [ ]:
sama_city_checked

,City,CITY_CD
2,abgeeg,243
3,al adhar,1068
4,al ahsa,3677
5,al haniyah,2496
6,al hawra,1218
...,...,...
14175,sulayyil,1210
14176,jeddah,18
14177,jeddah,18
14178,jeddah,18


In [ ]:
sama_city_checked.columns = ['city' , 'city_id']

In [ ]:
corpus.shape

(14300, 2)

In [ ]:
corpus = pd.concat([pos_city_country_final ,pos_dest_final ,saudi_cities , sama_city_checked])

In [ ]:
corpus.dropna(subset = ['city'] , inplace=True)

In [ ]:
corpus['city'] = corpus['city'].apply(lambda x: x.lower())

In [ ]:
corpus['city'] =corpus['city'] .replace('al madinah al munawwarah' , 'madina').replace('makkah al mukarramah' , 'makkah')

In [ ]:
corpus['city'] = corpus['city'].apply(lambda x: remove_prefix(x , 'al ')).apply(lambda x:remove_prefix(  x, 'al-'))

In [ ]:
corpus_city = list(set(corpus['city']))

In [ ]:
corpus_city

In [ ]:
#sama_city.drop(columns = ['CITY_CD' ,'CITY_NAME_EN'] , inplace=True)

In [ ]:
sama_city = sama_city.loc[sama_city['Checked'] !=1 , ]

In [ ]:
sama_city['City']

0                          
1                         -
24       jubail industrial 
40           rahwah al-bare
48            mahayel asser
                ...        
14167        central region
14168                others
14169             jordanian
14170             srilankan
14173                rorrma
Name: City, Length: 4218, dtype: object

In [ ]:
len(corpus_city)

4713

In [ ]:
sama_city['City'] = sama_city['City'].replace('al madinah al munawwarah' , 'madina').replace('makkah al mukarramah' , 'makkah')

In [ ]:
sama_city['City']

0                          
1                         -
24       jubail industrial 
40           rahwah al-bare
48            mahayel asser
                ...        
14167        central region
14168                others
14169             jordanian
14170             srilankan
14173                rorrma
Name: City, Length: 4218, dtype: object

In [ ]:
sama_city['City'] =sama_city['City'].apply(lambda x: remove_prefix(x , 'al ')).apply(lambda x:remove_prefix(  x, 'al-'))

In [ ]:
sama_city.head()

,Unnamed: 0,OperationId,City,CITY_CD,Checked
0,1,1,,,NaN
1,2,1,-,,NaN
24,25,1,jubail industrial,,NaN
40,41,2,rahwah al-bare,,NaN
48,49,2,mahayel asser,,NaN


In [ ]:
sama_city.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4218 entries, 0 to 14173
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   4218 non-null   int64  
 1   OperationId  4218 non-null   int64  
 2   City         4218 non-null   object 
 3   CITY_CD      4216 non-null   object 
 4   Checked      0 non-null      float64
dtypes: float64(1), int64(2), object(2)
memory usage: 197.7+ KB


In [ ]:
sama_city.reset_index( inplace=True)

In [ ]:
sama_city.dropna(subset=['City'] , inplace=True)

In [ ]:
sama_city['closest'] = sama_city['City'].apply(lambda x :  get_close_matches(x,corpus_city) )

In [ ]:
sama_city_done = sama_city.explode(column='closest').merge(how = 'left' ,
    right = corpus , left_on = 'closest' , right_on= 'city') 

In [ ]:
sama_city_done

,Unnamed: 0,OperationId,City,CITY_CD,Checked,closest,city,city_id
0,1,1,,,NaN,,,999999
1,1,1,,,NaN,,,999999
2,2,1,-,,NaN,NaN,NaN,NaN
3,25,1,jubail industrial,,NaN,NaN,NaN,NaN
4,41,2,rahwah al-bare,,NaN,hafr al-baten,hafr al-baten,47
...,...,...,...,...,...,...,...,...
80369,14174,7,rorrma,,NaN,sharora,sharora,3347
80370,14174,7,rorrma,,NaN,sharora,sharora,3347
80371,14174,7,rorrma,,NaN,sharora,sharora,3347
80372,14174,7,rorrma,,NaN,sharora,sharora,3347


In [ ]:
sama_city_done['a_d_comp'] =   sama_city_done.apply(sim_metric,
                          args=('City', 'city'),
                          axis=1) 

In [ ]:
sama_city_done.sort_values(by=['a_d_comp'] , ascending= False)

,Unnamed: 0,OperationId,City,city,city_id,a_d_comp
0,1,1,,,999999,1.0
69199,11971,6,,,999999,1.0
62355,10952,6,,,999999,1.0
4491,820,6,,,999999,1.0
37925,6404,4,,,999999,1.0
...,...,...,...,...,...,...
34951,5992,3,ern reg,NaN,NaN,0.0
75313,13079,3,offside,NaN,NaN,0.0
12880,2298,3,naseem owfee market,NaN,NaN,0.0
47308,8507,4,mecca,NaN,NaN,0.0


In [ ]:
sama_city_done=sama_city_done.loc[(sama_city_done.groupby(['Unnamed: 0'])['a_d_comp'].idxmax()) , 
                           ['Unnamed: 0' , 'OperationId'  ,'City' , 'city' ,'city_id' , 'a_d_comp']]

In [ ]:
sama_city_done['index'] +=1

In [ ]:
sama_city_done=sama_city_done.loc[sama_city_done['a_d_comp'] >= 0.8].sort_values(by=['a_d_comp'] , ascending=False)

In [ ]:
sama_city_done

,index,OperationId,City,city,city_id,a_d_comp
0,1,1,,,999999,1.0
30123,7354,2,riyadh,riyadh,3,1.0
30186,7368,2,kadarah,kadarah,3399,1.0
30179,7366,2,hajun,hajun,1505,1.0
30165,7363,2,khobar,khobar,31,1.0
...,...,...,...,...,...,...
10397,2514,6,sabia,sabya,3479,0.8
30534,7455,3,qaryat alu,qaryat al 'ulya,89,0.8
51882,12670,3,khamis al,khamis harb,1176,0.8
30502,7447,3,mahd,mahadi,2977,0.8


In [ ]:
sama_city_done.to_excel('sama_city_done.xlsx')